In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from mistralai import Mistral, DocumentURLChunk # DocumentFileChunk
from mistralai.extra import response_format_from_pydantic_model
import panel as pn

import ocular
from ocular.client import MistralOCRClient, RootModel, DocumentFileChunk

In [35]:
pn.extension()

In [3]:
json_filename = '../data/schema.json'

In [4]:
import json 
with open(json_filename, "r") as f:
    data = json.load(f)
    print(json.dumps(data, indent=4))

{
    "investimento_immobiliare": {
        "riassunto_perizia": "L'immobile oggetto di pignoramento \u00e8 un appartamento con cantina a Ravenna, con un valore di mercato stimato di circa \u20ac 157.726. Viene proposto all'asta a un prezzo base di \u20ac 137.317, offrendo un potenziale risparmio del 13%. Le principali criticit\u00e0 includono lievi difformit\u00e0 catastali sanabili (costo stimato \u20ac 5.000), un debito condominiale di \u20ac 3.573 e l'occupazione da parte del debitore esecutato. Esiste inoltre una potenziale controversia sulla propriet\u00e0 di una seconda cantina.",
        "identificativo": "Esecuzione Immobiliare R.G.E. N. 35/2024",
        "procedura_giudiziaria": {
            "tribunale": "Tribunale Civile di Ravenna",
            "atto_pignoramento": {
                "data": "04.03.2024",
                "numero_repertorio": "872",
                "dettagli_trascrizione": {
                    "data": "11.03.2024",
                    "registro_particolare"

In [5]:
client = MistralOCRClient()

{
  "$defs": {
    "AnalisiInvestimento": {
      "properties": {
        "potenziale_convenienza": {
          "$ref": "#/$defs/PotenzialeConvenienza"
        },
        "totale_costi_aggiuntivi_stimati": {
          "title": "Totale Costi Aggiuntivi Stimati",
          "type": "number"
        },
        "profilo_rischio": {
          "title": "Profilo Rischio",
          "type": "string"
        },
        "considerazioni_finali": {
          "title": "Considerazioni Finali",
          "type": "string"
        }
      },
      "required": [
        "potenziale_convenienza",
        "totale_costi_aggiuntivi_stimati",
        "profilo_rischio",
        "considerazioni_finali"
      ],
      "title": "AnalisiInvestimento",
      "type": "object"
    },
    "AttoPignoramento": {
      "properties": {
        "data": {
          "title": "Data",
          "type": "string"
        },
        "numero_repertorio": {
          "title": "Numero Repertorio",
          "type": "string"
        

In [6]:
file_path = '/home/emanuele/Downloads/RGE_RA_35_2024/RGE_RA_35 2024/PVP_4306904/Perizia di stima.PDF'

In [7]:
with open(file_path, 'rb') as file:
    file_content = file.read()

In [8]:
encoded_content = client._encode_file(file_content)

In [10]:
print("Uploading file to Mistral...")
uploaded_file = client.client.files.upload(
        
    file={
        "file_name": file_path,
        "content": file_content,
    },
    purpose="ocr",
)

Uploading file to Mistral...


In [14]:
# Get a signed URL for the uploaded file

print("Getting signed URL...")
signed_url = client.client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)
print(signed_url)

Getting signed URL...
url='https://mistralaifilesapiprodswe.blob.core.windows.net/fine-tune/8dd66733-5c0b-4495-a8a6-69a4e92f2b89/eb911111-cca9-4c3c-acbe-dd25d9521ccf/51a2512d7f604d8e8975632f1d636ffa.pdf?se=2025-08-28T10%3A24%3A26Z&sp=r&sv=2025-01-05&sr=b&sig=cLxZHSosFzTmAW3WVW7ribHl/DCoowJ8bYqceXeiScY%3D'


In [15]:
print("Processing local document with Mistral OCR...")

ocr_response = client.client.ocr.process(
    model="mistral-ocr-latest",
    document=DocumentURLChunk(document_url=signed_url.url),
    # response_format=response_format_from_pydantic_model(RootModel)
)

Processing local document with Mistral OCR...


In [16]:
# 7. Access the structured data
structured_data = ocr_response.parsed
print("\n✅ Document processed successfully!")
print("\nStructured JSON Output:")
print(structured_data.model_dump_json(indent=2))

AttributeError: 'OCRResponse' object has no attribute 'parsed'

In [21]:
type(ocr_response)

mistralai.models.ocrresponse.OCRResponse

In [29]:
data = json.loads(ocr_response.model_dump_json()) #()['pages'] #['markdown']
data.keys()

dict_keys(['pages', 'model', 'usage_info', 'document_annotation'])

In [46]:
data['pages'][0].keys()

dict_keys(['index', 'markdown', 'images', 'dimensions'])

In [41]:
text = ''.join([data['pages'][ix]['markdown'] for ix in range(len(data['pages']))])
#pn.pane.Markdown()

In [44]:
s = '''# REPUBBLICA ITALIANA 

## TRIBUNALE CIVILE DI RAVENNA

## RELAZIONE DI CONSULENZA TECNICA D’UFFICIO

ESECUZIONE IMMOBILIARE R.G.E. N. 35/2024
Giudice dell'Esecuzione:
DOTT. PAOLO GILOTTA
Sostituito in data 17.09.2024 da:
DOTT. MASSIMO VICINI

## DATI DI PIGNORAMENTO

Con atto di pignoramento immobili del 04.03.2024, Rep. ${ }^{\circ}$. 872, trascritto a Ravenna in data 11/03/2024 al Registro particolare n. 3398 e Registro generale n. 4550, sono stati pignorati i seguenti beni immobili: "porzione di fabbricato ad uso civile abitazione posto in Comune di Ravenna, via El Alamein n.2, costituita da un appartamento al piano terzo, con annessa cantina al piano terra, censita nel Catasto dei Fabbricati di detto Comune alla sezione RA Foglio 103 con particella 1714, sub 8, z.c. 1, cat. A/4, cl. 4, vani 6, R.C. euro 557,57".

## ํ๓๓

## CONSISTENZA, UBICAZIONE IMMOBILI
'''
pn.pane.Markdown(s)

<>:15: SyntaxWarning: invalid escape sequence '\c'
<>:15: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_2521114/1636064914.py:15: SyntaxWarning: invalid escape sequence '\c'
  Con atto di pignoramento immobili del 04.03.2024, Rep. ${ }^{\circ}$. 872, trascritto a Ravenna in data 11/03/2024 al Registro particolare n. 3398 e Registro generale n. 4550, sono stati pignorati i seguenti beni immobili: "porzione di fabbricato ad uso civile abitazione posto in Comune di Ravenna, via El Alamein n.2, costituita da un appartamento al piano terzo, con annessa cantina al piano terra, censita nel Catasto dei Fabbricati di detto Comune alla sezione RA Foglio 103 con particella 1714, sub 8, z.c. 1, cat. A/4, cl. 4, vani 6, R.C. euro 557,57".


Markdown(str)

In [42]:
print(text)

# REPUBBLICA ITALIANA 

## TRIBUNALE CIVILE DI RAVENNA

## RELAZIONE DI CONSULENZA TECNICA D’UFFICIO

ESECUZIONE IMMOBILIARE R.G.E. N. 35/2024
Giudice dell'Esecuzione:
DOTT. PAOLO GILOTTA
Sostituito in data 17.09.2024 da:
DOTT. MASSIMO VICINI

## DATI DI PIGNORAMENTO

Con atto di pignoramento immobili del 04.03.2024, Rep. ${ }^{\circ}$. 872, trascritto a Ravenna in data 11/03/2024 al Registro particolare n. 3398 e Registro generale n. 4550, sono stati pignorati i seguenti beni immobili: "porzione di fabbricato ad uso civile abitazione posto in Comune di Ravenna, via El Alamein n.2, costituita da un appartamento al piano terzo, con annessa cantina al piano terra, censita nel Catasto dei Fabbricati di detto Comune alla sezione RA Foglio 103 con particella 1714, sub 8, z.c. 1, cat. A/4, cl. 4, vani 6, R.C. euro 557,57".

## ํ๓๓

## CONSISTENZA, UBICAZIONE IMMOBILI

L'immobile a destinazione d'uso residenziale, oggetto del pignoramento, è sito nel Comune di Ravenna (Ra), al piano terzo di 

In [ ]:
parser = RobustUnstructuredParser(api_key="your-openai-api-key")

text = "John Smith, age 35, software developer at Google, lives in Mountain View, salary $150,000"

result = parser.parse_with_validation(
    text,
    expected_fields=["name", "age", "occupation"],
    schema_hint="Extract: name, age, occupation, company, location, salary"
)

if result.success:
    print(f"Parsed successfully (confidence: {result.confidence:.2f}):")
    print(json.dumps(result.data, indent=2))
else:
    print(f"Parsing failed: {result.error}")

In [47]:
# Quick usage
parser = UnstructuredToJSONParser()
result = parser.parse_to_json("Your unstructured text here")
print(json.dumps(result, indent=2))

NameError: name 'UnstructuredToJSONParser' is not defined